# Join, Concat e Pivot no Apache Spark

O Apache Spark fornece operações poderosas para unir, concatenar e pivotar DataFrames, permitindo a combinação de dados de várias fontes, a concatenação de DataFrames e a transformação de dados para análises mais avançadas.

## Operações de Join

As operações de join permitem combinar dois ou mais DataFrames com base em uma ou mais colunas de chave:

| Operação       | Descrição                                              | Exemplo de Caso de Uso                          |
|----------------|--------------------------------------------------------|-----------------------------------------------|
| `join`         | Combina DataFrames usando uma ou mais colunas de chave | Mesclar dados de tabelas relacionadas.         |
| `crossJoin`    | Realiza um join cruzado (cada linha com cada linha)    | Gerar todas as combinações possíveis.          |
| `leftJoin`     | Executa um join esquerdo (mantém todas as linhas do DataFrame esquerdo) | Unir dados de forma hierárquica.     |
| `rightJoin`    | Executa um join direito (mantém todas as linhas do DataFrame direito) | Unir dados de forma hierárquica.     |
| `fullOuterJoin` | Executa um join externo completo (mantém todas as linhas de ambos os DataFrames) | Unir dados preservando todos os registros. |

## Estratégias de Join no Apache Spark

| Estratégia        | Descrição                                                | Quando Usar                  | Desempenho        |
|-------------------|----------------------------------------------------------|------------------------------|-------------------|
| Shuffle Join      | Redistribuição de dados para realizar a junção           | Grandes DataFrames          | Custo e Tempo     |
| Broadcast Join    | Transmissão de um DataFrame pequeno para todos os nós   | Um DataFrame é pequeno     | Eficiente         |
| Sort-Merge Join   | Ordenação e fusão de DataFrames                          | DataFrames ordenados        | Eficiente         |
| Bucketed Join     | Otimização para DataFrames divididos em buckets         | Chave de junção frequente   | Melhora o desempenho |
| Broadcast Hash Join | Transmissão da tabela menor para todos os nós        | Uma tabela é pequena        | Melhora o desempenho |

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [ ]:
df_air_cia = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/air_cia/")
)

df_air_cia.show(1, vertical=True)

In [26]:
df_vra = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/vra/")
)
df_vra.show(1, vertical=True)

-RECORD 0-----------------------------------
 icao_empresa_aerea     | TAM               
 numero_voo             | 9458              
 codigo_di              | 0                 
 codigo_tipo_linha      | I                 
 icao_aerodromo_origem  | SBGR              
 icao_aerodromo_destino | SCEL              
 partida_prevista       | 01/01/2020 06:30  
 partida_real           | 01/01/2020 06:30  
 chegada_prevista       | 01/01/2020 10:40  
 chegada_real           | 01/01/2020 10:40  
 situacao_voo           | REALIZADO         
 codigo_justificativa   | null              
 preco_passagem         | 1907.3242         
 preco_combustivel      | 19443.44365734512 
only showing top 1 row



# inner, cross, outer, full, fullouter, full_outer, left, leftouter, left_outer, right, rightouter, right_outer, semi, leftsemi, left_semi, anti, leftanti and left_anti.

In [ ]:
df_air_cia.join(df_vra.hint("sortMerge"), df_vra["icao_empresa_aerea"] == df_air_cia["icao"],how="anti").show(10,vertical=True)

# Operações de Concatenação

As operações de concatenação permitem combinar DataFrames verticalmente (por linhas) ou horizontalmente (por colunas):

| Operação      | Descrição                                              | Exemplo de Caso de Uso                              |
|---------------|--------------------------------------------------------|---------------------------------------------------|
| `union`       | Concatena dois DataFrames verticalmente (por linhas)  | Combinar dados de DataFrames semelhantes.          |
| `unionAll`    | Concatena dois DataFrames verticalmente (por linhas)  | Combinar dados de DataFrames semelhantes.          |
| `unionByName` | Concatena dois DataFrames alinhando colunas pelo nome | Combinar dados de DataFrames com colunas renomeadas. |

In [7]:
df_vra_1 = df_vra.filter(df_vra["partida_real"] < "2022-07-01")
df_vra_2 = df_vra.filter(df_vra["partida_real"] >= "2022-07-01")

print(df_vra_1.count())
print(df_vra_2.count())

2725464
585540


In [17]:
3311004 == (2725464+585540)

True

In [16]:
df_vra_1.unionByName(df_vra_2, allowMissingColumns=True)

DataFrame[icao_empresa_aerea: string, numero_voo: string, codigo_di: string, codigo_tipo_linha: string, icao_aerodromo_origem: string, icao_aerodromo_destino: string, partida_prevista: timestamp, partida_real: timestamp, chegada_prevista: timestamp, chegada_real: timestamp, situacao_voo: string, codigo_justificativa: string, teste: int]

In [14]:
df_vra_2 = df_vra_2.select(df_vra_2.columns[::-1]).withColumn("teste", F.lit(1))